# Deriving and Implementing a Machine Learning Algorithm for Rating Melee Players

In this notebook, I derive a machine learning algorithm for rating (and thereby ranking) melee players based on head to head data. The underlying model is heavily inspired by TrueSkill, which is used by Xbox Live for matchmaking. See [Chapter 3 of Model-Based Machine Learning](http://www.mbmlbook.com/TrueSkill.html) for details. As with TrueSkill, this model can also be thought of as an improved version of the [Elo rating system.](https://en.wikipedia.org/wiki/Elo_rating_system)

The head to head data I used is the [2018 SSBM spreadsheet from GimmeDatWheat](https://twitter.com/gimmedatwheat/status/953346703381487617?s=21). Big thanks to them for collecting this data and making it publicly available. (On that note, I have no idea if this data is accurate or complete. If you have access to better data you'd like to share, please let me know.)

I'll start by giving the results and discussing their implications and limitations. Then I'll delve into the math and coding behind how I arrived at the results.

## The Results


In [11]:
with open("ssbm_results.txt", "r") as f:
    for line in f:
        print(line, end='')

Rank  Player            Score  
   1  Hungrybox          2822  
   2  Armada             2583  
   3  Leffen             2483  
   4  Plup               2476  
   5  Mango              2232  
   6  Mew2King           2191  
   7  Zain               2063  
   8  Wizzrobe           2052  
   9  SFAT               1877  
  10  Axe                1789  
  11  aMSa               1763  
  12  S2J                1757  
  13  Crush              1665  
  14  Westballz          1636  
  15  PewPewU            1561  
  16  ARMY               1539  
  17  KJH                1538  
  18  AbsentPage         1512  
  19  lloD               1510  
  20  Lucky              1501  
  21  Trif               1463  
  22  Swedish Delight    1439  
  23  n0ne               1436  
  24  Duck               1429  
  25  Shroomed           1372  
  26  Colbol             1330  
  27  La Luna            1285  
  28  ChuDat             1255  
  29  HugS               1233  
  30  iBDW               1229  
  31  Ju

### Interpretation and Limitations

The players are ranked by the 'Score' column, which is generated by my algorithm. There is some randomness inherent to the scores, so they shouldn't be over interpreted. For example, Leffen and Plup are basically tied as are n0ne and Swedish Delight. Zain and Wizzrobe are very close. 

Within the top 10, there is some clear stratification. Hungrybox is \#1 with a score over 200 points higher than the \#2, Armada. Armada's score is probably helped a lot by his winning record against Hungrybox. Armada in turn is 100 points ahead of Leffen and Plup, who are around 200 points ahead of Mango and Mew2King. Another 100 point gap separates Mango and Mew2King from Zain and Wizzrobe, who in turn are over 100 points higher than SFAT. I'm sure there's much more to be said about this ranking and scoring, and I'm curious to hear what other people think.

The algorithm implements a model that, like any model, makes many simplifying assumptions:

1. Each player has a single skill value represented by a real number. 
2. Each player has a performance value for each set, which varies from set to set, but on average is equal to the skill value for the player. 
3. The way the performance value varies about the skill value is the same for every player. 
4. The player with the larger performance value wins the set.
5. The outcome of each set is independent of the outcome of every other set.
6. All relevant sets are included.

Assuming this model, the algorithm assigns skill values to each player in such a way as to make the observed head to head records as likely as possible. Then, the standard deviations of the estimates of the skill values are calculated by bootstrapping the algorithm. The final score is the skill value minus two times the standard deviation, shifted and scaled to make the bottom score 0 and the top score ~3000. (This is a simplification. To see the full details, read below.)

The point of subtracting twice the standard deviation is to reward consistency, so that a player's estimated skill value doesn't come from just a couple big wins.

The assumptions reveal some obvious limitations to the model. I'm assuming that skill is a single number. To the extent that skill can be quantified at all, it must involve many numbers (e.g. corresponding to the different matchups), and it must evolve over time. Also the performance variability is the same for each player, whereas, for example, it's probably the case that ICs players are more likely than average to upset higher-skilled opponents. Finally, it is probably not the case that all relvant sets are included, since the data only includes 62 players, and only has their sets from the last year. Including more history runs into problems with assuming that skill is constant over time. Relaxing these assumptions could be the subject of future work.

## Theory

Now that you've seen the results, here's the model and the algorithm for fitting it. 

**Disclaimer**: This is some fairly technical content that you may not enjoy or understand without some background knowledge in machine learning. 

### The Probabilistic Model
The model is the following:  
We have $k$ players $p_1, \dots p_k$ each with a skill level $\theta_1, \dots, \theta_k$. 

When two players compete, the one with the higher skill is more likely to win, but they do not always win. We can imagine taking each player's skill level and adding to it some random noise to account for all the factors besides their true skill level. Whoever has the higher adjusted skill, after adding the noise, is the winner. Maybe the "better" player is tired, so they have some negative noise, and the other player is really hot that day, so they have some positive noise, and the player with a lower true skill wins on that day. 

We can assume that the noise is the same for all players (a big assumption) and that it takes very specific bell-shape, so that the sum of the noise follows a unit [logistic distribution](https://en.wikipedia.org/wiki/Logistic_distribution). Then following result holds:
$$
P(p_i \textrm{ beats } p_j | \theta_i, \theta_j) = \sigma(\theta_i - \theta_j),
$$
where $\sigma(x) = \frac{1}{1 + e^{-x}}$ is the [logistic function.](https://en.wikipedia.org/wiki/Logistic_function)

The logistic function squashes its inputs to be between 0 and 1, with large negative values getting mapped close to 0, 0 getting mapped to 1/2, and large positive values getting mapped close to 1. If you don't like that motivation in terms of adding noise to the true skill levels, then you can instead take it as given we're choosing to map differences in skill levels to probabilities of winning via the logistic function. I chose the logistic function because it makes the math and the coding easier than, say, the Gaussian CDF. Elo also uses the logistic function.

Let a *game* $g$ be a pair of indices $g = (g_1, g_2)$ such that $p_{g_1}$ beat $p_{g_2}$. Let $G$ be the collection of all games we have. Let $\vec{\theta} = (\theta_1, \dots, \theta_k)$ be the skill levels. Assuming the outcome of each game is independent,
$$
P(G|\vec{\theta}) = \prod_{g \in G} P(p_{g_1} \textrm{ beats } p_{g_2}|\vec{\theta}) = \prod_{g \in G} \sigma(\theta_{g_1} - \theta_{g_2}).
$$

We can fit this model via maximum likelihood. We can also assume an isotropic Gaussian prior $\vec{\theta} \sim \mathcal{N}(0, \sigma^2 I)$, and fit the model via maximum a posteriori to derive an [L2 penalized (weight decay)](https://en.wikipedia.org/wiki/Tikhonov_regularization) version. I won't show the regularization term in any of the derivations, but in the code below I add just a little regularization so the optimization is always well-posed. For example, without regularization the likelihood can always be increased by pushing the skill parameter lower for Kage because he has no wins.

### The Loss Function and its Gradient
To fit the model via maximum likelihood is the same as minimizing the negative log-likelihood. So we let the loss $J$ be the negative log-likelihood, modulo a factor of $1/n$ where $n = |G|$ is the number of games. That way $J$ is the average of the losses $J_g$ for each game $g$.
$$
J(\vec{\theta}) := -\frac{1}{n}l(\vec{\theta}; G) = -\frac{1}{n}\log(P(G|\vec{\theta})) = \frac{1}{n}\sum_{g \in G} -\log \sigma(\theta_{g_1} - \theta_{g_2}) =: \frac{1}{n}\sum_{g \in G} J_g(\vec{\theta})
$$

To miminize the loss $J$, we derive the gradient $\nabla J$. To derive the gradient, we just need the partial derivative of $J$ with respect to an arbitrary $\theta_j$. Also, since the overall loss $J$ is just the average of the per-game losses $J_g$, the derivative of $J$ is the average of the derivatives of $J_g$. In math,
$$
\nabla J = \frac{1}{n}\sum_{g \in G} \nabla J_g = 
\frac{1}{n}\sum_{g \in G} \left(\frac{\partial J_g}{\partial\theta_1}, \dots, \frac{\partial J_g}{\partial\theta_k}\right)
$$


So we just need to calculate the derivative of $J_g$.
$$
\frac{\partial}{\partial\theta_j} J_g(\vec{\theta}) =
\frac{\partial}{\partial\theta_j} -\log \sigma(\theta_{g_1} - \theta_{g_2}) =
\frac{-\frac{\partial}{\partial\theta_j} \sigma(\theta_{g_1} - \theta_{g_2})}{\sigma(\theta_{g_1} - \theta_{g_2})}
$$

Now we have three cases.  
1. If $j \neq g_1$ and $j \neq g_2$ then $J_g$ doesn't depend on $\theta_j$, so 
$$
\frac{\partial}{\partial\theta_j} J_g(\vec{\theta}) = 0.
$$
2. If $j = g_1$ (and therfore doesn't equal $g_2$ since people don't play against themselves), then 
$$
\frac{\partial}{\partial\theta_j} J_g(\vec{\theta}) =
\frac{-\frac{\partial}{\partial\theta_j} \sigma(\theta_{g_1} - \theta_{g_2})}{\sigma(\theta_{g_1} - \theta_{g_2})} = 
\frac{-\sigma(\theta_{g_1} - \theta_{g_2})(1 - \sigma(\theta_{g_1} - \theta_{g_2}))}{\sigma(\theta_{g_1} - \theta_{g_2})} = 
\sigma(\theta_{g_1} - \theta_{g_2}) - 1.
$$
3. If $j = g_2$ (and therefore doesn't equal $g_1$), we get the negation of case 2.
$$
\frac{\partial}{\partial\theta_j} J_g(\vec{\theta}) =
1 - \sigma(\theta_{g_1} - \theta_{g_2})
$$

### Remarks on the Math

Let's do a little sanity check on our derivation. The gradient points in the direction of steepest increase. So if we want to minimize our loss function, we should follow the direction of the negative gradient. If we observe a game where, say, player 1 beats player 2, then the *negative* derivative wrt $\theta_1$ of the loss for that game is $1 - \sigma(\theta_1 - \theta_2)$. So to decrease the loss we're supposed to adjust $\theta_1$ by $\varepsilon (1 - \sigma(\theta_1 - \theta_2))$. Since $\sigma(x) \in (0, 1)$ for all $x \in \mathbb{R}$, $1 - \sigma(\theta_1 - \theta_2) > 0$. That's good. It means if player 1 wins, we adjust their skill upwards. Moreover, if player 1 is already considered way better than player 2, so that $\theta_1 - \theta_2$ is large, then $\sigma(\theta_1 - \theta_2)$ will be close to 1, and $1 - \sigma(\theta_1 - \theta_2)$ will be close to zero. That makes sense. If we already expect player 1 to win easily, then we don't adjust player 1's skill much for winning. On the other hand, if player 1 is considered much worse than player 2, then the update will be large. Neat.

Similarly, the derivative wrt $\theta_2$ is just the oppositive of the derivative wrt $\theta_1$, so player 2's skill level gets adjusted downwards by the same amount that player 1's skill goes up. 

There's a nice conservation of skill in these gradient-based updates. This is very similar to how rating moves around under the Elo system. However, in Elo, each game only "counts once" and only affects the ratings of the players involved. This leads to problems like players being able to "sit on" their rating by not playing any games. In my algorithm, every game keeps getting used to move skill around until we reach an optimal configuration of skills that best explains the observed head to head records. When a new game is played, to update the skills we'd have to run the whole algorithm again (a huge computational disadvantage). But this means that the new game can potentially affect everyone's skills, not just the two players who played the game. This way, a player who doesn't play any games can still lose skill. Also, under the ranking system described with the results, players who play less games have less certain skill estimates and get punished in the rankings.

### Optimizing the Loss

Using the gradient we derived above, we can optimize the loss function via gradient descent, or any of its fancier momentum-based siblings like RMSprop or adam. Also, since the overall loss takes the form of an average of individual losses, we can use stochastic (aka minibatch) gradient descent. I don't think we'll really have any use for a fancy optimizer or minibatches. Vanilla gradient descent should work just fine.

### Implementing the Algorithm

Below I use numpy to get a rough version of the model working quickly. I tried to map the math directly onto the code. So I represent the collection of all games as an $n \times 2$ array and call it `G`. Each row represents one game, and the left entry of the row is the index of the winning player, while the right entry is the index of the losing player. The skill levels are held in a 1d array called `theta`. The amount of regularization is controlled by `lam` for $\lambda$ with `lam = 0` representing no regularization. 

The representation of the games is kind of weird and inefficient, since if player 1 beats player 2 1000 times, it seems wasteful to record that as 1000 separate rows of an array, instead of one row with the number 1000 in it somewhere. Nevertheless, with the relatively small amount of data I'm working with this isn't a big issue.

In [1]:
import numpy as np
from scipy.special import expit #expit is logistic function

# the loss function
def J(theta, G, lam):
    return np.mean(-np.log(expit(theta[G[:, 0]] - theta[G[:, 1]]))) + lam / 2 * np.dot(theta, theta)

Now we do a little sanity check on the loss function.

In [2]:
# player 0 beats player 1 twice
# player 1 beats player 2 twice
G = np.array([(0, 1), (0, 1), (1, 2), (1, 2)])

# skills seem appropriate, low loss
print(J(np.array([6, 3, 1]), G, 0))

# skill seem backwards, high loss
print(J(np.array([1, 3, 6]), G, 0))

0.08775768130835727
2.587757681308357


Now I implement the gradient of the loss and compare the analytic gradient against the numerical gradient. The cell below shows just one test, but I tried various numbers and it looks like the gradient is correct.

In [3]:
# gradient of the loss
def gradJ(theta, G, lam):
    n = G.shape[0]
    grad = np.zeros((n, theta.shape[0]))
    winner_partials = expit(theta[G[:, 0]] - theta[G[:, 1]]) - 1
    grad[np.arange(n), G[:, 0]] = winner_partials
    grad[np.arange(n), G[:, 1]] = -winner_partials
    grad = np.mean(grad, axis=0)
    return grad + lam * theta

def numerical_gradJ(theta, G, lam, h=1e-5):
    k = theta.shape[0]
    grad = np.zeros(k)
    for i in range(k):
        ei = np.zeros(k)
        ei[i] = 1
        grad[i] = (J(theta + h * ei, G, lam) - J(theta - h * ei, G, lam)) / (2 * h)
    return grad

# compute the difference between the two gradients. Should be super small
theta = np.array([1, 3, 6])
np.abs(numerical_gradJ(theta, G, 0.2) - gradJ(theta, G, 0.2))

array([4.19647928e-11, 4.27965441e-11, 7.65476571e-11])

Now that I'm reasonably confident the code is working correctly, I'll write a function for fitting the model. `fit` fits the model via vanilla gradient descent. `G` is the games array, `k` is how many players there are, `theta` is the initial value for theta (defaults to all zeros), `step` is the step size for gradient descent, `iterations` is the number of gradient descent steps, `verbose` controls whether the loss gets printed on each iteration. `verbose` is useful for diagnosing convergence.

The defaults for `step` and `iteration` were chosen by tinkering with the data and observing the loss over each iteration. `step` can be quite large because the gradient is always between -1 and 1, and is usually very small because of the averaging I do in the loss function. The default for `lam` was chosen to be small enough to not really affect the majority of the estimates too much, but it's still positive so as to always make the problem well-posed.

In [4]:
def fit(G, k, lam=1e-4, theta=None, step=50, iterations=100, verbose=False):
    if not theta:
        theta = np.zeros(k)
    for i in range(iterations):
        if verbose:
            print(f"Iteration {i}: Loss = {J(theta, G, lam)}")
        theta -= step * gradJ(theta, G, lam)
    return theta

### Simulation test

So far so good. Now I write some simple code to generate data from the model given a true `theta` vector. The `n` parameter controls how many games each pair of players plays against each other. Then I generate a large data set and see if the algorithm can recover the true values. As I remarked above, the gradient-based updates conserve skill, so the sum of all the skill in the system will always be zero if we initialize the skill vector to all zeros. There's no problem of how much skill "there really is" since the model only deals with differences in skills, and is therefore invariant to shifting all the skill levels by the same amount. So without loss of generality, we can assume there is zero total skill. 

Below, you can see that the `fit` function approximately recovers the `true_theta`.

In [5]:
from random import random
def generate_data(theta, n):
    G = []
    for i in range(len(theta)):
        for j in range(i + 1, len(theta)):
            prob_i_wins = expit(theta[i] - theta[j])
            for _ in range(n):
                if random() < prob_i_wins:
                    G.append((i, j))
                else:
                    G.append((j, i))
    return np.array(G)

# sums to 0
true_theta = [-3, -2, 1, 4]
G = generate_data(true_theta, 2000)
fit(G, 4, step=10)

array([-3.02532518, -2.06742979,  1.05139475,  4.04136022])

### Loading and Cleaning the Data

As I mentioned above, I got the head to head data from [GimmeDatWheat's Google Sheet.](https://docs.google.com/spreadsheets/d/1G6ffVQMrnojT2xpkxUJxz30djdjbQBDQYzIx4dlBU-I/edit#gid=388132110) I exported the sheet as a csv file. After a bunch of trial and error I came up with the code below for transforming the data into the weird array format my algorithm requires. I also extract the first three columns to get the 2017 and 2018 ranks and a list of all the player names.

In [6]:
with open("2018 SSBM - Top 50 H2H.csv", "r") as f:
    x = f.readlines()
rows = [line.split(',')[:-1] for line in x[1:]]
rank2017 = [row[0] for row in rows]
rank2018 = [row[1] for row in rows]
names = [row[2] for row in rows]
G = []
for i, row in enumerate(rows):
    for record in zip(range(i+1, len(names)), rows[i][5+2*i::2], rows[i][6+2*i::2]):
        if record[1]:
            for _ in range(int(record[1])):
                G.append((i, record[0]))
            for _ in range(int(record[2])):
                G.append((record[0], i))
G = np.array(G)

### Fitting the model
Now we can finally fit the model! Below I fit the model and printed out a table of the results. The rows are ordered by 'score' which is the skill rating from the model. The 2018 and 2017 rank columns come from the original Google Sheet. 

A couple of things stand out. Firstly, DruggedFox has a skill rating of exactly 0. If you look in the sheet, you'll see he has no sets recorded. So his skill never got updated, and he never influenced anyone else's skill. We should probably just remove him. In a related vein, there are other players who have very few sets recorded in the data, but this is not reflected anywhere in the ranking. For example Laudandus only has 4 sets in the data, but he gets ranked 36 for his one win over Nintendude and his one win over AbsentPage.

We could try to incorporate uncertainty in the model through some ad hoc device like removing players with too few sets or penalizing their rating somehow based on the number of games they've played. But if a player only plays a few games, we might know their skill fairly precisely, given that we know the skills of their opponents very precisely. So it would be good to have a more sophisticated approach to assessing the uncertainty in our skill estimates. If we did have such an uncertainty estimate, then we could rank the players by subtracting the uncertainty from the skill estimate or something like that.

In [7]:
# some code for nicely printing a table
def print_table(tbl, header=None):
    if header:
        tbl = [header] + tbl
    ncols = len(tbl[0])
    widths = []
    for i in range(ncols):
        widths.append(max(len(str(row[i])) for row in tbl))
    for row in tbl:
        for i in range(ncols):
            print(f"{row[i]:{widths[i]}}", end='  ')
        print()

skills = fit(G, len(names))
tbl = sorted(zip(names, skills.round(3)), key=lambda x: -x[1])
tbl = [(i+1,) + row for i, row in enumerate(tbl)]
print_table(tbl, header=('Rank', 'Player', 'Score'))

Rank  Player            Score   
   1  Hungrybox          2.898  
   2  Armada               2.6  
   3  Leffen             2.291  
   4  Plup               2.222  
   5  Mango              1.808  
   6  Mew2King           1.766  
   7  Wizzrobe           1.318  
   8  Zain               1.261  
   9  aMSa               0.917  
  10  Axe                0.916  
  11  SFAT               0.818  
  12  Crush              0.693  
  13  S2J                0.607  
  14  PewPewU            0.427  
  15  Westballz          0.396  
  16  Trif               0.299  
  17  lloD               0.216  
  18  AbsentPage         0.211  
  19  KJH                  0.2  
  20  Colbol             0.116  
  21  ARMY               0.114  
  22  Shroomed           0.113  
  23  Lucky                0.1  
  24  Swedish Delight    0.087  
  25  n0ne               0.055  
  26  Duck               0.055  
  27  DruggedFox           0.0  
  28  Santiago          -0.022  
  29  Bananas           -0.035  
  30  ChuD

### Assessing Uncertainty with the Bootstrap

[The bootstrap](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) is one of the foundational techniques of modern statistics. In many common scenarios, it allows us to use computational power instead of mathematical derivations to understand the uncertainty in our estimates. The basic idea is that we sample from the data set we do have to generate many different data sets. If our data set is like the "population" we want to understand, then the resampled data sets are like new data sets from the population. Then by computing estimates for each resampled data set we can see what kind of variability the estimation procedure has across different data sets. Of course, the data set we have is not the true population. So the boostrap does not give us the true uncertainty. It just gives us a decent estimate. 

To be able to apply the bootstrap, you have to have the time to run your procedure thousands of times over. Luckily, this just takes a few minutes for the code we have here. If you were doing something like deep learning, where it can take hours just to train one model, you'd be out of luck.

What I'll do specifically is resample the sets themselves. Then I'll fit the model on the different resampled data sets and calculate the standard deviation of each player's estimated skill across all the resamples. If someone's estimated skill depends heavily on just a few sets, when we fit the model on different resampled data sets that sometimes have those sets and sometimes don't, their skill will move all over the place. So that person will have a high standard deviation, and they will be penalized in the rankings.

In [8]:
def boot(G, k, replicates=100, lam=1e-4, theta=None, step=10, iterations=10):
    n = G.shape[0]
    boots = np.zeros((replicates, k))
    for i in range(replicates):
        ix = np.random.randint(n, size=n)
        G_star = G[ix, :]
        boots[i, :] = fit(G_star, k, lam, theta, step, iterations)
    return boots

# try different numbers of replicates until the estimates seem relatively stable
sds1 = np.std(boot(G, len(names), replicates=5000, step=50, iterations=100), axis=0)
sds2 = np.std(boot(G, len(names), replicates=5000, step=50, iterations=100), axis=0)
print(np.mean(np.abs(sds1 - sds2)))

0.004309251285388446


Now we've got the standard deviations of our estimates. I propose to score the players by their estimated skill minus two times their standard deviation. This is similar to what TrueSkill does to rank players, except TrueSkill uses three standard deviations. The idea is that the players will now be ranked such that their skill rating is most likely at least as high as their displayed score. This way consistency is rewarded. Also, I removed DruggedFox, since he actually doesn't get punished for having no games at all. His score is always 0.

Finally, I shifted and scaled the scores to make the lowest score zero and the highest score around 3000, since that's what people are used to because of Elo.

In [9]:
score = skills - 2 * sds1
score -= np.min(score)
score *= 500
score = score.astype(np.int64)
tbl = sorted(zip(names, score), key=lambda x: -x[1])
tbl = [(player, score) for player, score in tbl if player != 'DruggedFox']
tbl = [(rank + 1, player, score) for rank, (player, score) in enumerate(tbl)]
print_table(tbl, header=('Rank', 'Player', 'Score'))

Rank  Player            Score  
   1  Hungrybox          2822  
   2  Armada             2583  
   3  Leffen             2483  
   4  Plup               2476  
   5  Mango              2232  
   6  Mew2King           2191  
   7  Zain               2063  
   8  Wizzrobe           2052  
   9  SFAT               1877  
  10  Axe                1789  
  11  aMSa               1763  
  12  S2J                1757  
  13  Crush              1665  
  14  Westballz          1636  
  15  PewPewU            1561  
  16  ARMY               1539  
  17  KJH                1538  
  18  AbsentPage         1512  
  19  lloD               1510  
  20  Lucky              1501  
  21  Trif               1463  
  22  Swedish Delight    1439  
  23  n0ne               1436  
  24  Duck               1429  
  25  Shroomed           1372  
  26  Colbol             1330  
  27  La Luna            1285  
  28  ChuDat             1255  
  29  HugS               1233  
  30  iBDW               1229  
  31  Ju

In [10]:
# write the output to a file so I can include it at the top
def write_table(f, tbl, header=None):
    if header:
        tbl = [header] + tbl
    ncols = len(tbl[0])
    widths = []
    for i in range(ncols):
        widths.append(max(len(str(row[i])) for row in tbl))
    for row in tbl:
        for i in range(ncols):
            f.write(f"{row[i]:{widths[i]}}  ")
        f.write('\n')
        
with open("ssbm_results.txt", "w") as f:
    write_table(f, tbl, header=('Rank', 'Player', 'Score'))